# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [2]:
!pip install faiss-cpu


In [3]:
# ✅ Install SerpAPI Python client
!pip install google-search-results


In [4]:
from typing import Optional
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain_community.utilities import SerpAPIWrapper
import faiss
import os

In [5]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [6]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import userdata

# ✅ Retrieve and manually inject the secret into environment
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY")

# ✅ Confirm
print("OPENAI_API_KEY is set:", os.environ.get("OPENAI_API_KEY") is not None)
print("SERPAPI_API_KEY is set:", os.environ.get("SERPAPI_API_KEY") is not None)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OPENAI_API_KEY is set: True
SERPAPI_API_KEY is set: True


In [18]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [19]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

# A TODO tool — helps plan survival actions
todo_prompt = PromptTemplate.from_template(
    "You are a prepper AI expert. Create a survival to-do list for: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

# Web search tool
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Survival Search",
        func=search.run,
        description="Useful for finding up-to-date survival tips or emergency procedures online.",
    ),
    Tool(
        name="Survival Planner",
        func=todo_chain.run,
        description="Use this to create survival plans or checklists. Input: an objective to survive. Output: a detailed checklist.",
    ),
]


In [20]:
prefix = """You are a survivalist AI performing a task based on the following objective: {objective}.
Consider these previously completed tasks: {context}."""

suffix = """Task: {task}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)


In [26]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True  # <- add this
)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [27]:
OBJECTIVE = "Create a beginner’s guide to surviving your first week in a zombie apocalypse"

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=False,
    max_iterations=3  # Keep it short and observable
)

baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...

Question: What should be included in a todo list for surviving the first week of a zombie apocalypse?
Thought: It's important to prioritize tasks and have a plan in place.
Action: Survival Planner
Action Input: Objective - Surviving the first week of a zombie apocalypse
Observation: 

1. Secure Shelter: The first step in surviving a zombie apocalypse is finding a safe place to stay. Look for a sturdy building with minimal entry points, such as a house with a fenced yard or a high-rise apartment with secure entrances.

2. Gather Supplies: Stock up on essential supplies such as food, water, and medical supplies. Aim for at least a week's worth of provisions, but try to gather as much as possible. Don't forget to include tools, weapons, and other survival gear.

3. Create a Plan: Develop a plan for how you will survive the first week of the zombie apocalypse. This s

{'objective': 'Create a beginner’s guide to surviving your first week in a zombie apocalypse'}

In [23]:
def reset_baby_agi():
    index = faiss.IndexFlatL2(1536)
    vectorstore = FAISS(
        OpenAIEmbeddings().embed_query,
        index,
        InMemoryDocstore({}),
        {}
    )
    return BabyAGI.from_llm(
        llm=llm,
        vectorstore=vectorstore,
        task_execution_chain=agent_executor,
        verbose=True,
        max_iterations=3
    )

baby_agi = reset_baby_agi()


In [ ]:
# OBJECTIVE = "Write a weather report for SF today"

In [ ]:
# # Logging of LLMChains
# verbose = False
# # If None, will keep on going forever
# max_iterations: Optional[int] = 3
# baby_agi = BabyAGI.from_llm(
#     llm=llm,
#     vectorstore=vectorstore,
#     task_execution_chain=agent_executor,
#     verbose=verbose,
#     max_iterations=max_iterations,
# )

In [ ]:
# baby_agi({"objective": OBJECTIVE})